[View in Colaboratory](https://colab.research.google.com/github/kimitakem/colab_ex/blob/master/sample.ipynb)

# Tensorflow Eager executiomモードのチュートリアル要約
## 初期化：
- ライブラリのインポートと、Eagerモードのアクティベーションを行う


In [0]:
import tensorflow as tf
tf.enable_eager_execution()

## 環境の確認
- Tensorflowのバージョン
- Eagerモードになっているかの確認

In [0]:
import tensorflow.contrib.eager as tfe

print("Tensorflow Version: {}".format(tf.__version__))
print("Check Eager Mode: {}".format(tf.executing_eagerly()))




Tensorflow Version: 1.9.0
Check Eager Mode: True


## 微分演算を行う

In [0]:
tfe = tf.contrib.eager

w = tfe.Variable([[5.0]])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[10.]], shape=(1, 1), dtype=float32)


## 単純モデルの学習
- Mnist
